In [31]:
#Imports of all necessary Python Packages
import sklearn as skl
import numpy as np
import pandas as pd

#Classifiers
from sklearn.svm import SVC


In [26]:
#Here import the data to classify
data = np.random.rand(100,5)
labels = np.random.randint(2,size=100)

In [32]:
classf = SVC()